# PyCity Schools Analysis

* As a whole, schools with higher budgets, did not yield better test results. By contrast, schools with higher spending per student actually (645-675) underperformed compared to schools with smaller budgets (585 per student). This effect could be confounded by school size as it seems smaller schools had smaller budgets per student, further analyses are required.

* Up until 2000 students in size, schools do not see variance in student testing outcomes, differences only appear at the large categories of student body size(2000+)

* As a whole, charter schools out-performed the public district schools across all metrics. Particularly interesting is the divergence between test scores and passing rates, close average test scores and disproportional test passing rates indicate that students of charter schools have smaller variance in their scores compared to students from district schools. 
---

In [34]:
# Dependencies and Setup
import pandas as pd
import numpy as np

# File to Load (Remember to Change These)
school_data_to_load = "Resources/schools_complete.csv"
student_data_to_load = "Resources/students_complete.csv"

# Read School and Student Data File and store into Pandas Data Frames
school_data = pd.read_csv(school_data_to_load)
student_data = pd.read_csv(student_data_to_load)

# Combine the data into a single dataset
school_data_complete = pd.merge(student_data, school_data, how="left", on=["school_name", "school_name"])

In [35]:
#get df overview
school_data_complete.head()

,Student ID,student_name,gender,grade,school_name,reading_score,math_score,School ID,type,size,budget
0,0,Paul Bradley,M,9th,Huang High School,66,79,0,District,2917,1910635
1,1,Victor Smith,M,12th,Huang High School,94,61,0,District,2917,1910635
2,2,Kevin Rodriguez,M,12th,Huang High School,90,60,0,District,2917,1910635
3,3,Dr. Richard Scott,M,12th,Huang High School,67,58,0,District,2917,1910635
4,4,Bonnie Ray,F,9th,Huang High School,97,84,0,District,2917,1910635


In [36]:
#get total school number
total_schools = school_data["School ID"].count()
#get total student number
total_students = school_data["size"].sum()
#get total budget
total_budget = school_data["budget"].sum()
#---------------------------
#get average math score
avg_math_score = school_data_complete["math_score"].mean()
#get average reading score
avg_reading_score = school_data_complete["reading_score"].mean()
#--------------------------

#get passing math rate
# Find students with math score at/over 70
math_passing = school_data_complete.loc[school_data_complete["math_score"] >= 70,  ["Student ID"]]
math_passes = len(math_passing["Student ID"])
math_passing_rate = (math_passes / total_students)*100

#get passing reading rate
# Find students with reading score at/over 70
reading_passing = school_data_complete.loc[school_data_complete["reading_score"] >= 70,  ["Student ID"]]
reading_passes = len(reading_passing["Student ID"])
reading_passing_rate = (reading_passes / total_students)*100

#get overall passing rate
overall_pass_rate = ((math_passes + reading_passes) / (total_students*2))*100

#assign pass or fail
school_data_complete["math_pass"] = [100 if x >= 70 else 0 for x in school_data_complete["math_score"]]

school_data_complete["reading_pass"] = [100 if x >= 70 else 0 for x in school_data_complete["reading_score"]]

In [37]:
district_summary_dict = {
    "Total Schools"         : total_schools, 
    "Total Students"        : total_students,
    "Total Budget"          : '${:,.2f}'.format(total_budget),
    "Average Math Score"    : round(avg_math_score, 2),
    "Average Reading Score" : round(avg_reading_score, 2),
    "% Passing Math"        : round(math_passing_rate, 2),
    "% Passing Reading"     : round(reading_passing_rate, 2),
    "Overall Passing Rate"  : round(overall_pass_rate, 2)
}

district_summary = pd.DataFrame(district_summary_dict, index=[0])
district_summary

,Total Schools,Total Students,Total Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,Overall Passing Rate
0,15,39170,"$24,649,428.00",78.99,81.88,74.98,85.81,80.39


In [38]:
#copy of school data
per_school_data = school_data

# Group average data by school
grouped_complete = school_data_complete.groupby(school_data_complete['school_name']).mean().reset_index()

In [41]:
# Calculate per student budget
budget_per_student = grouped_complete["budget"]/ grouped_complete["size"]
grouped_complete["budget_per_student"] = budget_per_student
grouped_complete["Overall passing rate"] = ((grouped_complete["math_pass"] + grouped_complete["reading_pass"])/2)

#merge dfs
merged_complete = pd.merge(school_data, grouped_complete, how = "left", on = "school_name")

In [42]:
#drop duplicate columns
merged_clean = merged_complete.drop(columns=['size_x', 'budget_x', 'Student ID',
                                             'School ID_y', 'School ID_x'], axis=1)
merged_clean = merged_clean.set_index('school_name')
merged_clean

,type,reading_score,math_score,size_y,budget_y,math_pass,reading_pass,budget_per_student,Overall passing rate
school_name,,,,,,,,,
Huang High School,District,81.182722,76.629414,2917.0,1910635.0,65.683922,81.316421,655.0,73.500171
Figueroa High School,District,81.158020,76.711767,2949.0,1884411.0,65.988471,80.739234,639.0,73.363852
Shelton High School,Charter,83.725724,83.359455,1761.0,1056600.0,93.867121,95.854628,600.0,94.860875
Hernandez High School,District,80.934412,77.289752,4635.0,3022020.0,66.752967,80.862999,652.0,73.807983
Griffin High School,Charter,83.816757,83.351499,1468.0,917500.0,93.392371,97.138965,625.0,95.265668
Wilson High School,Charter,83.989488,83.274201,2283.0,1319574.0,93.867718,96.539641,578.0,95.203679
Cabrera High School,Charter,83.975780,83.061895,1858.0,1081356.0,94.133477,97.039828,582.0,95.586652
Bailey High School,District,81.033963,77.048432,4976.0,3124928.0,66.680064,81.933280,628.0,74.306672
Holden High School,Charter,83.814988,83.803279,427.0,248087.0,92.505855,96.252927,581.0,94.379391


In [43]:
#top 5 passing rate
top5 = merged_clean.sort_values(by = ['Overall passing rate'], ascending=False)
top5.head()

,type,reading_score,math_score,size_y,budget_y,math_pass,reading_pass,budget_per_student,Overall passing rate
school_name,,,,,,,,,
Cabrera High School,Charter,83.975780,83.061895,1858.0,1081356.0,94.133477,97.039828,582.0,95.586652
Thomas High School,Charter,83.848930,83.418349,1635.0,1043130.0,93.272171,97.308869,638.0,95.290520
Pena High School,Charter,84.044699,83.839917,962.0,585858.0,94.594595,95.945946,609.0,95.270270
Griffin High School,Charter,83.816757,83.351499,1468.0,917500.0,93.392371,97.138965,625.0,95.265668
Wilson High School,Charter,83.989488,83.274201,2283.0,1319574.0,93.867718,96.539641,578.0,95.203679


In [44]:
#bottom 5 passing rate
bottom5 = merged_clean.sort_values(by = ['Overall passing rate'])
bottom5.head()

,type,reading_score,math_score,size_y,budget_y,math_pass,reading_pass,budget_per_student,Overall passing rate
school_name,,,,,,,,,
Rodriguez High School,District,80.744686,76.842711,3999.0,2547363.0,66.366592,80.220055,637.0,73.293323
Figueroa High School,District,81.158020,76.711767,2949.0,1884411.0,65.988471,80.739234,639.0,73.363852
Huang High School,District,81.182722,76.629414,2917.0,1910635.0,65.683922,81.316421,655.0,73.500171
Johnson High School,District,80.966394,77.072464,4761.0,3094650.0,66.057551,81.222432,650.0,73.639992
Ford High School,District,80.746258,77.102592,2739.0,1763916.0,68.309602,79.299014,644.0,73.804308


## Math Scores by Grade

* Create a table that lists the average Reading Score for students of each grade level (9th, 10th, 11th, 12th) at each school.

  * Create a pandas series for each grade. Hint: use a conditional statement.
  
  * Group each series by school
  
  * Combine the series into a dataframe
  
  * Optional: give the displayed data cleaner formatting

In [45]:
#separate by grade
by_grade = school_data_complete.groupby(['school_name','grade']).mean().reset_index().round(2)

#math and reading separate tables

math_by_grade = by_grade.pivot(index='school_name', columns ='grade', values='math_score')

math_by_grade

grade,10th,11th,12th,9th
school_name,,,,
Bailey High School,77.00,77.52,76.49,77.08
Cabrera High School,83.15,82.77,83.28,83.09
Figueroa High School,76.54,76.88,77.15,76.40
Ford High School,77.67,76.92,76.18,77.36
Griffin High School,84.23,83.84,83.36,82.04
Hernandez High School,77.34,77.14,77.19,77.44
Holden High School,83.43,85.00,82.86,83.79
Huang High School,75.91,76.45,77.23,77.03
Johnson High School,76.69,77.49,76.86,77.19


In [46]:
reading_by_grade = by_grade.pivot(index='school_name', columns ='grade', values='reading_score')

reading_by_grade

grade,10th,11th,12th,9th
school_name,,,,
Bailey High School,80.91,80.95,80.91,81.30
Cabrera High School,84.25,83.79,84.29,83.68
Figueroa High School,81.41,80.64,81.38,81.20
Ford High School,81.26,80.40,80.66,80.63
Griffin High School,83.71,84.29,84.01,83.37
Hernandez High School,80.66,81.40,80.86,80.87
Holden High School,83.32,83.82,84.70,83.68
Huang High School,81.51,81.42,80.31,81.29
Johnson High School,80.77,80.62,81.23,81.26


## Scores by School Spending

In [74]:
# Sample bins. Feel free to create your own bins.
spending_bins = [0, 585, 615, 645, 675]
bin_names = ["<$585", "$585-615", "$615-645", "$645-675"]

spending_merged_clean = merged_clean.drop(columns=['type', 'size_y', 'budget_y'])

spending_merged_clean["Spending level(p/std)"] = pd.cut(spending_merged_clean["budget_per_student"],
                                                        bins=spending_bins, labels=bin_names)

table_price = spending_merged_clean.groupby(["Spending level(p/std)"]).mean().reset_index().round(2)
table_price = table_price.drop(columns=['budget_per_student'])
table_price.set_index("Spending level(p/std)")

,reading_score,math_score,math_pass,reading_pass,Overall passing rate
Spending level(p/std),,,,,
<$585,83.93,83.46,93.46,96.61,95.04
$585-615,83.89,83.60,94.23,95.90,95.07
$615-645,81.89,79.08,75.67,86.11,80.89
$645-675,81.03,77.00,66.16,81.13,73.65


## Scores by School Size

* Perform the same operations as above, based on school size.

In [73]:
# Sample bins. Feel free to create your own bins.
size_bins = [0, 1000, 2000, 5000]
school_sizes = ["Small (<1000)", "Medium (1000-2000)", "Large (2000-5000)"]

spending_merged_clean = merged_clean.drop(columns=['type', 'budget_per_student', 'budget_y'])

spending_merged_clean["School size"] = pd.cut(spending_merged_clean["size_y"],
                                                        bins=size_bins, labels=school_sizes)

table_size = spending_merged_clean.groupby(["School size"]).mean().reset_index().round(2)
table_size = table_size.drop(columns=['size_y'])
table_size.set_index("School size")


,reading_score,math_score,math_pass,reading_pass,Overall passing rate
School size,,,,,
Small (<1000),83.93,83.82,93.55,96.10,94.82
Medium (1000-2000),83.86,83.37,93.60,96.79,95.20
Large (2000-5000),81.34,77.75,69.96,82.77,76.36


## Scores by School Type

In [82]:
spending_merged_clean = merged_clean.drop(columns=['size_y','budget_per_student', 'budget_y'])

table_type = spending_merged_clean.groupby(["type"]).mean().reset_index().round(2)
table_type.set_index("type")

,reading_score,math_score,math_pass,reading_pass,Overall passing rate
type,,,,,
Charter,83.90,83.47,93.62,96.59,95.10
District,80.97,76.96,66.55,80.80,73.67
